# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import re
import pandas as pd
from sqlalchemy import create_engine
import pickle
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, make_scorer
#classifiers
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV

In [41]:
from sklearn.base import BaseEstimator
class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = RandomForestClassifier(),
        ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [2]:
# load data from database
engine = create_engine('sqlite:///message.db')
df = pd.read_sql_table('messages',engine)
df.shape

(26345, 40)

In [8]:
#prepare feature and classes as arrays
X = df['message'].values
Y = df[df.columns[4:40]].values

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ",text)

    # tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [71]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(estimator=DecisionTreeClassifier()))
    ])     

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .30, random_state=42)

In [12]:
pipeline.fit(X_train, Y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000023DC9211B80>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
Y_pred=pipeline.predict(X_test)

In [20]:
f1_score=[]
precision=[]
classif=[]
for n in range(Y_pred.shape[1]):
    try:
        s=classification_report(Y_test[:,n], Y_pred[:,n],output_dict=True)
        classif.append(df.columns[n+4])
        precision.append(s['1.0']['precision'])
        f1_score.append(s['1.0']['f1-score'])
    except:
        print(df.columns[n+4]+' does not have a 1')
        continue
results=pd.DataFrame(classif,columns=['classifier'])
results['precision']=precision
results['f1-score']=f1_score
results['model']="KNeighborsClassifier"

print('avg_precision:'+str(sum(precision)/len(precision))+', avg_f1-score:'+str(sum(f1_score)/len(f1_score)))

avg_precision:0.18710006718911312, avg_f1-score:0.0962346784100256


In [24]:
results.to_excel("results.xlsx", index=False)

### 6. Improve your model
Use grid search to find better parameters. 

In [72]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x0000023DC9211B80>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=DecisionTreeClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x0000023DC9211B80>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=DecisionTreeClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,


In [77]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),    
        ('clf', MultiOutputClassifier(estimator=ClfSwitcher())),
])

parameters = [
    {
        'clf__estimator': [DecisionTreeClassifier()],       
        'clf__estimator': [GaussianProcessClassifier()],
        'clf__estimator': [RandomForestClassifier()],
        'clf__estimator': [MLPClassifier()]
    }
]

scorers = {
    'precision_score': make_scorer(precision),
    'f1_score': make_scorer(f1_score)
}

In [ ]:
cv = GridSearchCV(pipeline, parameters, return_train_score=scorers, cv=3, verbose=3)
cv.fit(X, Y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] clf__estimator=MLPClassifier() ..................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
pd.DataFrame(cv.cv_results_).to_excel("2021_03_15_run2.xlsx", index=False)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
# Open the file to save as pkl file
model_pkl = open(model_filepath, 'wb')
pickle.dump(model, model_pkl)
# Close the pickle instances
model_pkl.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.